In [1]:
import pandas as pd
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import numpy as np

In [2]:
!pip freeze > requirements.txt

In [ ]:
import pandas as pd

# Charger le dataset
# Load your datasets
df1 = pd.read_csv("/content/DataSetQ&A2.csv",encoding="Windows-1252",sep=';')
df2 = pd.read_csv("/content/batch_1_a10 (1).csv")
df3 = pd.read_csv("/content/batch_1_a8.csv")
df4 = pd.read_csv("/content/batch_2_a6 (1).csv")
df5 = pd.read_csv("/content/batch_2_a8.csv")
df6 = pd.read_csv("/content/batch_3_a7.csv")
df7 = pd.read_csv("/content/batch_3_a9.csv")
df8 = pd.read_csv("//content/batch_4_a11.csv")
df9 = pd.read_csv("/content/batch_4_a9.csv")
df10 = pd.read_csv("/content/batch_1_a10.csv")
df11 = pd.read_csv("/content/batch_2_a6.csv")


# Select only the columns you need
df1_selected = df1[['question', 'answer']]
df2_selected = df2[['question', 'answer']]
df3_selected = df3[['question', 'answer']]
df4_selected = df4[['question', 'answer']]
df5_selected = df5[['question', 'answer']]
df6_selected = df6[['question', 'answer']]
df7_selected = df7[['question', 'answer']]
df8_selected = df8[['question', 'answer']]
df9_selected = df9[['question', 'answer']]
df10_selected = df10[['question', 'answer']]
df11_selected = df11[['question', 'answer']]

# Concatenate the selected columns into one dataframe
df=pd.concat([df1_selected, df2_selected, df3_selected,df4_selected,df5_selected,df6_selected,df7_selected,df8_selected,df9_selected,df10_selected,df11_selected], ignore_index=True)



In [ ]:
df = df.dropna()

In [ ]:
# Charger le dataset
questions = df["question"].tolist()
answers = df["answer"].tolist()

In [ ]:
# Charger le modèle et le tokenizer T5
tokenizer = T5Tokenizer.from_pretrained("t5-Base")
model = TFT5ForConditionalGeneration.from_pretrained("t5-Base")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
# Préparer les entrées et les sorties
inputs = [f"question: {q} </s>" for q in questions]
outputs = [f"{a} </s>" for a in answers]

In [ ]:
# Tokeniser les entrées et sorties
input_encodings = tokenizer(inputs, padding=True, truncation=True, return_tensors="tf")
output_encodings = tokenizer(outputs, padding=True, truncation=True, return_tensors="tf")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [ ]:
# Convertir les données en TensorFlow Dataset
input_ids = input_encodings['input_ids']
attention_mask = input_encodings['attention_mask']
labels = output_encodings['input_ids']

In [ ]:

# Prepare the dataset to include labels directly
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels},
))

# Prepare the dataset for training
train_dataset = train_dataset.shuffle(1000).batch(4)

# Définir l'optimiseur
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# Fonction de perte
def compute_loss(labels, logits):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))


In [ ]:
# Entraîner le modèle
model.compile(optimizer=optimizer, loss=compute_loss)

# Entraîner le modèle sur 20 époques
model.fit(train_dataset, epochs=30)


Epoch 1/30
314/314 [==============================] - 451s 1s/step - loss: 0.6733
Epoch 2/30
314/314 [==============================] - 392s 1s/step - loss: 0.2823
Epoch 3/30
314/314 [==============================] - 392s 1s/step - loss: 0.2569
Epoch 4/30
314/314 [==============================] - 392s 1s/step - loss: 0.2404
Epoch 5/30
314/314 [==============================] - 392s 1s/step - loss: 0.2275
Epoch 6/30
314/314 [==============================] - 392s 1s/step - loss: 0.2164
Epoch 7/30
314/314 [==============================] - 392s 1s/step - loss: 0.2059
Epoch 8/30
314/314 [==============================] - 392s 1s/step - loss: 0.1954
Epoch 9/30
314/314 [==============================] - 392s 1s/step - loss: 0.1858
Epoch 10/30
314/314 [==============================] - 391s 1s/step - loss: 0.1764
Epoch 11/30
314/314 [==============================] - 392s 1s/step - loss: 0.1671
Epoch 12/30
314/314 [==============================] - 392s 1s/step - loss: 0.1591
Epoch 13/30
3

In [ ]:
def generate_answer(question):
    input_text = f"question: {question} </s>"
    input_ids = tokenizer(input_text, return_tensors="tf").input_ids

    # Generate response with parameters to increase length and detail
    outputs = model.generate(
        input_ids,
        max_length=100,  # Increase to allow longer responses
        min_length=50,   # Set minimum length for longer responses
        num_beams=5,     # Increase beams for higher quality output
        do_sample=True,  # Enable sampling for diversity
        temperature=0.5, # Adjust temperature for creative responses
        top_p=0.9        # Nucleus sampling for diversity
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer



# Exemple d’utilisation
print(generate_answer("What are good food during pregnancy"))

Foods rich in iron and protein, such as lean meats, lentils, yogurt, salmon, nuts, vegetables, and whole grains, can help support both your health and your baby’s development, allowing you to focus on enjoying your pregnancy more effectively.


In [ ]:
from google.colab import drive

# Mount Google Drive to access the drive's files and directories
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_directory = "/content/drive/MyDrive/fine_tuned_T5PregnancyChatbot"
model.save_pretrained(save_directory)